In [1]:
from DataHandler.data import  *
from DataHandler.mapping import  *
from DataHandler.preprocessing import  *
from ModelCode.model import *

import json
import random
import os
import time

from lime.lime_text import LimeTextExplainer

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

from DataHandler.data import  *
from DataHandler.mapping import  *

from tqdm import tqdm
from tqdm import tqdm_notebook
import pickle

In [19]:
params={
      'dataset':'Dataset/jigsaw-toxic-severity-rating/validation_data.csv',
      'model':'tfidf-reddit-val',
      'cache_path':'../../Saved_models/',
      'model_path':'tfidf',
      'random_seed':2021,
      'save_path':'Saved_Models/',
      'logging':'local'  
       }

In [20]:
filename = 'Saved_Models/' + params['model'] + '/finalized_model.sav'
tfidf = pickle.load(open(filename, 'rb'))


In [21]:
test_df = pd.read_csv(params['dataset'])

less = test_df['less_toxic'].copy()
more = test_df['more_toxic'].copy()
# less = [clean_text(text) for text in less]
# more = [clean_text(text) for text in more]


In [22]:
test_df['less_score'] = tfidf.predict(less)
test_df['more_score'] = tfidf.predict(more)
test_df['diff'] = test_df['less_score'] - test_df['more_score']

In [23]:
# test_df.drop(columns = ['worker'], inplace=True)
test_df

,worker,less_toxic,more_toxic,less_score,more_score,diff
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,0.386273,0.470183,-0.083911
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,0.447289,0.451530,-0.004241
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",0.574796,0.519697,0.055099
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,0.480101,0.605897,-0.125796
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",0.464583,0.644866,-0.180283
...,...,...,...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,0.614552,0.357963,0.256590
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,0.614552,0.357963,0.256590
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,0.437005,0.513746,-0.076741
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,0.437005,0.513746,-0.076741


In [24]:
test_df.to_csv('Predicitions/' + params['model'] + '.csv', index=False)

In [25]:
misclass = test_df[test_df['diff'] >= 0]

In [26]:
misclass.sort_values('diff', ascending=False, inplace=True)

<ipython-input-26-cd547b7b836f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misclass.sort_values('diff', ascending=False, inplace=True)


In [27]:
misclass.drop_duplicates(subset=['less_toxic', 'more_toxic'], keep='first', inplace=True)

<ipython-input-27-04daaf1f665e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misclass.drop_duplicates(subset=['less_toxic', 'more_toxic'], keep='first', inplace=True)


In [28]:
misclass = misclass.head(200)

In [29]:
misclass.to_csv('Top200Misclassified/' + params['model'] + '.csv', index=False)